In [24]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyMuPDFLoader

In [26]:
# 1. 도구 정의
# ================================================================================

# 1. PDF 문서 검색 도구
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader('data/SPRI_AI_Brief_2023년12월호_F.pdf')
docs = loader.load()

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [27]:
from langchain.tools.retriever import create_retriever_tool
from langchain_core.prompts import PromptTemplate

# 어떤 정보들을 리트리버 검색 결과에 포함시켜 줘야 될지 지정합니다. 
document_prompt = PromptTemplate.from_template(                         
    "<document><content>{page_content}</content><page>{page}</page><filename>{source}</filename></document>"
)

retriever_tool = create_retriever_tool(     # retriever를 도구로 정의
    retriever,                              # 리트리버
    name='pdf_search',                      # 도구의 이름
    description='use this tool to search for information in the PDF file',       # 도구에 대한 설명을 자세히 기입해야 합니다. 
    document_prompt=document_prompt,
)

In [28]:
retriever_tool.invoke('삼성전자가 개할한 생성형 AI는?')

'<document><content>SPRi AI Brief |  \n2023-12월호\n10\n삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개\nn 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 \nAI 모델 ‘삼성 가우스’를 공개\nn 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 \n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유\nKEY Contents\n£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원\nn 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 \n‘삼성 가우스’를 최초 공개\n∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 \n최적화된 크기의 모델 선택이 가능\n∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, \n온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유\n∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 \n단계적으로 탑재할 계획\nn 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 \n이미지 모델의 3개 모델로 구성\n∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 \n처리를 지원\n∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 \n사내 소프트웨어 개발에 최적화\n∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원하는 대로 바꿀 수 있도록 지원하며 \n저해상도 이미지의 고해상도 전환도 지원\nn IT 전문지 테크리퍼블릭(TechRepublic)은 온디바이스 AI가 주요 기술 트렌

In [29]:
from langchain_community.tools.tavily_search import TavilySearchResults

# 2. search 도구

search  = TavilySearchResults(k=6)      # 검색 결과를 6개까지 가져오겠다

In [30]:
from langchain_community.agent_toolkits import FileManagementToolkit

# 3. 파일 관리 도구

if not os.path.exists('tmp5'):                              # 작업 폴더 생성
    os.mkdir('tmp5')

working_directory = 'tmp5'                                  # 작업 디렉토리 설정 

file_tools  = FileManagementToolkit(                        # 파일 관리 도구를 가져온다
    root_dir=str(working_directory),
    selected_tools=['write_file', 'read_file', 'list_directory']
).get_tools()

In [31]:
tools = file_tools + [retriever_tool]

In [32]:
for tool in tools:                                      
    print(f' - {tool.name} : {tool.description}')

 - write_file : Write file to disk
 - read_file : Read file from disk
 - list_directory : List files and directories in a specified folder
 - pdf_search : use this tool to search for information in the PDF file


In [33]:
from langchain_openai import ChatOpenAI

# 4. LLM 정의

llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

In [34]:
from langchain_core.prompts import ChatPromptTemplate

# 5. 프롬프트 생성

prompt = ChatPromptTemplate.from_messages(                  # 프롬프트
    [
        (
            'system',
            'You are a helpful assistant. '
            'You are a professional researcher. '
            'You can use the pdf_search tool to search for information in the PDF file. '
            'You can find further information by using search tool. '
            'You can use image generation tool to generate image from text. '
            'Finally, you can use file management tool to save your research result into files.'
        ),
        ('placeholder', '{chat_history}'),
        ('human', '{input}'),
        ('placeholder', '{agent_scratchpad}'),
    ]
)

In [35]:
from langchain.agents import create_tool_calling_agent

# 6. agent 정의

agent = create_tool_calling_agent(llm, tools, prompt)       # tool calling agent 생성

In [36]:
from langchain.agents import AgentExecutor

# 7.AgentExecutor 정의

agent_executor = AgentExecutor(     
    agent=agent,                    # 각 단계에서 계획을 생성하고 행동을 결정하는 agent
    tools=tools,                    # agent 가 사용할 수 있는 도구 목록
    verbose=True,
    max_iterations=10,              # 최대 10번 까지 반복
    max_execution_time=10,          # 실행되는데 소요되는 최대 시간
    handle_parsing_errors=True      
)

In [37]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# 8. 채팅 기록을 위한 메모리

store = {}   

def get_session_history(session_ids):   # session_id 를 기반으로 세션 기록을 가져오는 함수    
    if session_ids not in store:        # session_id 가 store에 없는 경우        
        store[session_ids] = ChatMessageHistory()   # 새로운 ChatMessageHistory 객체를 딕셔너리에 저장
    
    return store[session_ids]           # 해당 세션 ID에 대한 세션 기록 리턴


agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,                                    # session_id 를 기반으로 세션 기록을 가져오는 함수
    input_messages_key='input',                             # 프롬프트의 질문으로 들어가는 key: input
    history_messages_key='chat_history'                     # 프롬프트의 대화 기록이 들어가는 key: chat_history
)

In [38]:
result = agent_with_chat_history.stream(
    {
        "input": "삼성전자가 개발한 `생성형 AI` 와 관련된 유용한 정보들을 PDF 문서에서 찾아서 bullet point로 정리해 주세요. "
        "한글로 작성해주세요."
        "다음으로는 `report.md` 파일을 새롭게 생성하여 정리한 내용을 저장해주세요. \n\n"
        "#작성방법: \n"
        "1. markdown header 2 크기로 적절한 제목을 작성하세요. \n"
        "2. 발췌한 PDF 문서의 페이지 번호, 파일명을 기입하세요(예시: page 10, filename.pdf). \n"
        "3. 정리된 bullet point를 작성하세요. \n"
        "4. 작성이 완료되면 파일을 `report.md` 에 저장하세요. \n"
        "5. 마지막으로 저장한 `report.md` 파일을 읽어서 출력해 주세요. \n"
    },
    config={"configurable": {"session_id": "abc123"}},
)

In [39]:
for step in result:

    if 'actions' in step:
        print('==' * 50)
        print(f'[actions]')
        # print(step['actions'])
        print('[도구 호출]')
        print(f"Tool : {step['actions'][0].tool}")
        print(f"query : {step['actions'][0].tool_input}")
        print(f"Log: ", end='')
        print(f"{step['actions'][0].log}")
        # print(f"message log: {step['actions'][0].message_log}")
        print('==' * 50)

    elif 'steps' in step:
        print('\n')
        print('==' * 50)
        print(f'[steps]')
        # print(step['steps'])
        print(f"검색어: {step['steps'][0].action.tool_input}")
        # print(f"message log: {step['steps'][0].action.message_log}")
        print(f"[관찰 내용]")
        print(f"Observation:")
        print(f"{step['steps'][0].observation}")
        print('==' * 50)

    elif 'output' in step:
        print('==' * 50)
        # print(f"[output]: {step['output']}")
        print(f"[output]")
        print(f"step['messages']:")
        print(f"{step['messages'][0].content}")
        print('==' * 50)



> Entering new None chain...
[actions]
[도구 호출]
Tool : pdf_search
query : {'query': '삼성전자 생성형 AI'}
Log: 
Invoking: `pdf_search` with `{'query': '삼성전자 생성형 AI'}`




Invoking: `pdf_search` with `{'query': '삼성전자 생성형 AI'}`


<document><content>SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기

Stopping agent prematurely due to triggering stop condition


[actions]
[도구 호출]
Tool : read_file
query : {'file_path': 'report.md'}
Log: 
Invoking: `read_file` with `{'file_path': 'report.md'}`




Invoking: `read_file` with `{'file_path': 'report.md'}`


## 삼성전자의 생성형 AI ‘삼성 가우스’ 관련 정보

- 페이지: 10, 12
- 파일명: SPRI_AI_Brief_2023년12월호_F.pdf

- 삼성전자가 자체 개발한 생성형 AI 모델 ‘삼성 가우스’를 공개하였음.
- 삼성 가우스는 언어, 코드, 이미지의 3개 모델로 구성되어 있으며, 온디바이스에서 작동 가능.
- 온디바이스 작동으로 인해 사용자 정보 유출 위험이 없음.
- 삼성 가우스는 다양한 제품에 단계적으로 탑재될 계획임.
- 언어 모델은 메일 작성, 문서 요약, 번역 업무를 지원.
- 코드 모델 ‘코드아이(code.i)’는 대화형 인터페이스로 사내 소프트웨어 개발에 최적화됨.
- 이미지 모델은 창의적인 이미지 생성 및 저해상도 이미지를 고해상도로 전환하는 기능을 제공.
- 2024년부터 삼성 스마트폰에 가우스가 탑재될 예정이며, 메타의 라마2 및 구글 픽셀과 경쟁할 것으로 예상됨.

[steps]
검색어: {'file_path': 'report.md'}
[관찰 내용]
Observation:
## 삼성전자의 생성형 AI ‘삼성 가우스’ 관련 정보

- 페이지: 10, 12
- 파일명: SPRI_AI_Brief_2023년12월호_F.pdf

- 삼성전자가 자체 개발한 생성형 AI 모델 ‘삼성 가우스’를 공개하였음.
- 삼성 가우스는 언어, 코드, 이미지의 3개 모델로 구성되어 있으며, 온디바이스에서 작동 가능.
- 온디바이스 작동으로 인해 사용자 정보 유출 위험이 없음.
- 삼성 가우스는 다양한 제품에 단계적으로 탑재될 계획임.
- 언어 모델은 메일 작성, 문서 요약, 번역 업무를 지원.
- 코드 모델 ‘

In [40]:
def tool_callback(tool) -> None:
    print(f'===== 도구 호출 =====')
    print(f"사용된 도구 이름 : {tool[0]}")
    print(f'===== 도구 호출 =====')

In [41]:
def obervation_callback(observation) -> None:
    print(f'\n===== 관찰 내용 =====')
    print(f"Observation: {observation[0].observation}")
    print(f'===== 관찰 내용 =====')

In [42]:
def result_callback(result: str) -> None:
    print(f'===== 최종 답변 =====')
    print(f"{result}")
    print(f'===== 최종 답변 =====')

In [43]:
# 웹 검색을 통해 보고서 파일 업데이트
result = agent_with_chat_history.stream(
    {
        "input": "이번에는 삼성전자가 개발한 `생성형 AI` 와 관련된 정보들을 웹 검색하고, 검색한 결과를 정리해 주세요. "
        "한글로 작성해주세요."
        "다음으로는 `report.md` 파일을 열어서 기존의 내용을 읽고, 웹 검색하여 찾은 정보를 이전에 작성한 형식에 맞춰 뒷 부분에 추가해 주세요. \n\n"
        "#작성방법: \n"
        "1. markdown header 2 크기로 적절한 제목을 작성하세요. \n"
        "2. 정보의 출처(url)를 기입하세요(예시: 출처: 네이버 지식백과). \n"
        "3. 정리된 웹검색 내용을 작성하세요. \n"
        "4. 작성이 완료되면 파일을 `report.md` 에 저장하세요. \n"
        "5. 마지막으로 저장한 `report.md` 파일을 읽어서 출력해 주세요. \n"
    },
    config={"configurable": {"session_id": "abc123"}},
)

In [44]:
for step in result:
    if 'actions' in step:
        tool_callback(step['actions'])
    elif 'steps' in step:
        obervation_callback(step['steps'])
    elif 'output' in step:
        result_callback(step['messages'][0].content)  



> Entering new None chain...
===== 도구 호출 =====
사용된 도구 이름 : tool='read_file' tool_input={'file_path': 'report.md'} log="\nInvoking: `read_file` with `{'file_path': 'report.md'}`\n\n\n" message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_2Vy8JPTA87WIUAVdVwMJwD3F', 'function': {'arguments': '{"file_path":"report.md"}', 'name': 'read_file'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_13eed4fce1'}, id='run-76d7ca21-4340-4080-a6b7-ee6df4d3c70f', tool_calls=[{'name': 'read_file', 'args': {'file_path': 'report.md'}, 'id': 'call_2Vy8JPTA87WIUAVdVwMJwD3F', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'read_file', 'args': '{"file_path":"report.md"}', 'id': 'call_2Vy8JPTA87WIUAVdVwMJwD3F', 'index': 0, 'type': 'tool_call_chunk'}])] tool_call_id='call_2Vy8JPTA87WIUAVdVwMJwD3F'
===== 도구 호출 =====

Invoking: `read_file` with `{'file_path': 'report.md'}`



In [45]:
# 보고서 작성을 요청합니다.
result = agent_with_chat_history.stream(
    {
        "input": "`report.md` 파일을 열어서 안의 내용을 출력하세요. "
        "출력된 내용을 바탕으로, 전문적인 수준의 보고서를 작성하세요. "
        "보고서는 총 3개의 섹션으로 구성되어야 합니다:\n"
        "1. 개요: 보고서 abstract 를 300자 내외로 작성하세요.\n"
        "2. 핵심내용: 보고서의 핵심 내용을 작성하세요. 정리된 표를 markdown 형식으로 작성하여 추가하세요. "
        "3. 최종결론: 보고서의 최종 결론을 작성하세요. 출처(파일명, url 등)을 표시하세요."
        "마지막으로 작성된 결과물을 `report-2.md` 파일에 저장하세요."
    },
    config={"configurable": {"session_id": "abc123"}},
)

In [46]:
for step in result:
    if 'actions' in step:
        tool_callback(step['actions'])
    elif 'steps' in step:
        obervation_callback(step['steps'])
    elif 'output' in step:
        result_callback(step['messages'][0].content)  



> Entering new None chain...
===== 도구 호출 =====
사용된 도구 이름 : tool='read_file' tool_input={'file_path': 'report.md'} log="\nInvoking: `read_file` with `{'file_path': 'report.md'}`\n\n\n" message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_KHUBj3wFAK1q2vg8GqGDwJyI', 'function': {'arguments': '{"file_path":"report.md"}', 'name': 'read_file'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_13eed4fce1'}, id='run-aa7f9ef4-bd0b-450f-b42c-77fd1d562b9e', tool_calls=[{'name': 'read_file', 'args': {'file_path': 'report.md'}, 'id': 'call_KHUBj3wFAK1q2vg8GqGDwJyI', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'read_file', 'args': '{"file_path":"report.md"}', 'id': 'call_KHUBj3wFAK1q2vg8GqGDwJyI', 'index': 0, 'type': 'tool_call_chunk'}])] tool_call_id='call_KHUBj3wFAK1q2vg8GqGDwJyI'
===== 도구 호출 =====

Invoking: `read_file` with `{'file_path': 'report.md'}`

